In [1]:
!pip install jaro-winkler
!pip install scikit-learn
!pip install torcheval

In [2]:
import torch
from torcheval.metrics.functional import binary_f1_score
from sklearn.metrics import precision_score, recall_score
import json
from jaro import jaro_winkler_metric

def read_lines(file: str) -> list[str]:
    """Read lines from a file and return as a list."""
    with open(file, "r") as f:
        lines = f.readlines()
    return lines

def read_jsonl(file: str) -> list[list[tuple[str]]]:
    """Read JSON lines from a file and parse into a list of lists of tuples."""
    lines = read_lines(file)
    result = [[tuple(triplet) for triplet in json.loads(line)] for line in lines]
    return result

def read_txt(file: str) -> list[str]:
    """Read lines from a text file and return as a list."""
    return read_lines(file)

def to_binary(preds, gt):
    """Convert prediction and ground truth samples to binary classification."""
    max_len = max(len(preds), len(gt))
    output = [1 if pred in gt else 0 for pred in preds]
    target = [1 if gt in preds else 0 for gt in gt]

    # Pad the shorter tensor with zeros
    if len(output) < max_len:
        output.extend([0] * (max_len - len(output)))
    if len(target) < max_len:
        target.extend([0] * (max_len - len(target)))

    return torch.tensor(output), torch.tensor(target)

def calculate_scores(preds, gt):
    """Calculate F1 score, precision, recall, and Jaro-Winkler score."""
    max_len = max(len(gt), len(preds))
    f1_score_sum = precision_sum = recall_sum = jw_score_sum = 0

    for i in range(max_len):
        # Calculate F1 score, precision, and recall
        pred_sample = preds[min(i, len(preds) - 1)]
        gt_sample = gt[min(i, len(gt) - 1)]

        # Convert samples to lowercase
        pred_sample = [[item.lower() for item in triplet] for triplet in pred_sample]
        gt_sample = [[item.lower() for item in triplet] for triplet in gt_sample]

        output, target = to_binary(pred_sample, gt_sample)
        output, target = torch.tensor(output), torch.tensor(target)
        f1_score_sum += binary_f1_score(output, target)
        precision_sum += precision_score(target, output)
        recall_sum += recall_score(target, output)

        # Calculate Jaro-Winkler score
        pred_sample_txt = ' '.join([' '.join(triplet) for triplet in pred_sample])
        gt_sample_txt = ' '.join([' '.join(triplet) for triplet in gt_sample])
        jw_score_sum += jaro_winkler_metric(pred_sample_txt, gt_sample_txt)

    # Calculate averages
    f1_score = f1_score_sum / max_len
    precision = precision_sum / max_len
    recall = recall_sum / max_len
    jw_score = jw_score_sum / max_len

    return float(f1_score), precision, recall, jw_score

c:\Users\navde\AppData\Local\Programs\Python\Python310\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [4]:
import json
import statistics
from tqdm import tqdm
from json import JSONDecodeError
import re

def extract_tags(text):
    # Regular expression to match substrings enclosed within <>
    pattern = r'<([^<]*?)>(.*?)<\/\1>'

    # Find all tagged substrings
    tagged_substrings = re.findall(pattern, text)

    # Return each tag and content separately
    tagged_lists = [[tag, content] for tag, content in tagged_substrings]

    return tagged_lists

# def extract_tags(text):
#     # Regular expression pattern to match substrings enclosed within <>
#     pattern = r'<[^<]*?>.*?<\/[^<]*?>'

#     # Find all tagged substrings
#     tagged_substrings = re.findall(pattern, text)

#     # Split each tagged substring into individual tags and content
#     tagged_lists = []
#     for substring in tagged_substrings:
#         # Extract individual tags and content
#         tags_and_content = re.findall(r'<([^>]+)>(.*?)<\/\1>', substring)
#         tagged_lists.extend(tags_and_content)

#     return tagged_lists



successful_output = 0

# Create lists
f1_score_list = []
precision_list = []
recall_list = []
jw_score_list = []
successful_output = 0
failed_output = 0

test_set_path = 'C:/Users/navde/Desktop/Bioinformatics_401_Project/Data/Tagging_50_Test.jsonl'
truth_file_path = 'C:/Users/navde/Desktop/Bioinformatics_401_Project/Data/Tagging_Ground_Truth_Bias.jsonl'

with open(test_set_path, 'r',encoding="utf-8") as infile, open(truth_file_path, "r",encoding="utf-8") as outfile:
    for line, gt_line in tqdm(zip(infile, outfile)):
        successful_output += 1
        data = json.loads(line)
        datagt = json.loads(gt_line)
        abstract = data['input']
        ground_truth = datagt['input']

        print(extract_tags(abstract))
        print(extract_tags(ground_truth))

        # Get scores
        f1_score, precision, recall, jw_score = calculate_scores(extract_tags(abstract), extract_tags(ground_truth))

        #Increment variables
        f1_score_list.append(f1_score)
        precision_list.append(precision)
        recall_list.append(recall)
        jw_score_list.append(jw_score)

# Calculate statistics for each list
for name, score_list in [("F1 Score", f1_score_list), ("Precision", precision_list), ("Recall", recall_list), ("Jaro-Winkler Score", jw_score_list)]:
    avg_score = sum(score_list) / successful_output if successful_output > 0 else 0
    min_score = min(score_list) if score_list else 0
    max_score = max(score_list) if score_list else 0
    std_dev = statistics.stdev(score_list) if len(score_list) > 1 else 0  # Std. dev. requires at least 2 data points

    # Print statistics
    print(f"{name}:")
    print(f"  Average: {avg_score}")
    print(f"  Minimum: {min_score}")
    print(f"  Maximum: {max_score}")
    print(f"  Standard Deviation: {std_dev}")

# Print other information
total_outputs = successful_output + failed_output
print(f"Successful Outputs: {successful_output}")
print(f"Failed Outputs: {failed_output}")
print(f"Total Outputs: {total_outputs}")

# Write results to a file
with open("Tagger/results_of_tagger_bias_2.txt", "w") as f:
    for name, score_list in [("F1 Score", f1_score_list), ("Precision", precision_list), ("Recall", recall_list), ("Jaro-Winkler Score", jw_score_list)]:
        avg_score = sum(score_list) / successful_output if successful_output > 0 else 0
        min_score = min(score_list) if score_list else 0
        max_score = max(score_list) if score_list else 0
        std_dev = statistics.stdev(score_list) if len(score_list) > 1 else 0  # Std. dev. requires at least 2 data points

        # Write statistics to file
        f.write(f"{name}:\n")
        f.write(f"  Average: {avg_score}\n")
        f.write(f"  Minimum: {min_score}\n")
        f.write(f"  Maximum: {max_score}\n")
        f.write(f"  Standard Deviation: {std_dev}\n")

    # Write other information
    total_outputs = successful_output + failed_output
    f.write(f"Successful Outputs: {successful_output}\n")
    f.write(f"Failed Outputs: {failed_output}\n")
    f.write(f"Total Outputs: {total_outputs}\n")

print("Results have been written to results_of_tagger.txt.")


0it [00:00, ?it/s]

C:\Users\navde\AppData\Local\Temp\ipykernel_9356\3267912990.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output, target = torch.tensor(output), torch.tensor(target)
C:\Users\navde\AppData\Local\Temp\ipykernel_9356\3267912990.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output, target = torch.tensor(output), torch.tensor(target)
C:\Users\navde\AppData\Local\Temp\ipykernel_9356\3267912990.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output, target = torch.tensor(output), torch.tensor(target)
C:\Users\navde\AppData\Lo

[['source', 'Tartary Buckwheat'], ['source', 'Tartary buckwheat'], ['organoleptic_effect', 'rich'], ['source', 'Tartary buckwheat'], ['source', 'Tartary buckwheat'], ['source', 'Saskatoon <organoleptic_effect>berry</organoleptic_effect>'], ['source', 'Tartary buckwheat'], ['organoleptic_effect', 'acid'], ['CHEMICAL', '7-methylsulfinylheptyl glucosinolate'], ['source', 'Tartary buckwheat'], ['source', 'Saskatoon <organoleptic_effect>berry</organoleptic_effect>'], ['source', 'Tartary buckwheat'], ['source', 'Tartary buckwheat'], ['organoleptic_effect', 'rich'], ['source', 'human']]
[['source', 'Tartary Buckwheat'], ['source', 'Tartary buckwheat'], ['organoleptic_effect', 'rich'], ['source', 'Tartary buckwheat'], ['source', 'Tartary buckwheat'], ['source', 'Saskatoon <organoleptic_effect>berry</organoleptic_effect>'], ['source', 'Tartary buckwheat'], ['organoleptic_effect', 'acid'], ['CHEMICAL', '7-methylsulfinylheptyl glucosinolate'], ['source', 'Tartary buckwheat'], ['source', 'Saskatoo

C:\Users\navde\AppData\Local\Temp\ipykernel_9356\3267912990.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output, target = torch.tensor(output), torch.tensor(target)
C:\Users\navde\AppData\Local\Temp\ipykernel_9356\3267912990.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output, target = torch.tensor(output), torch.tensor(target)
C:\Users\navde\AppData\Local\Temp\ipykernel_9356\3267912990.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output, target = torch.tensor(output), torch.tensor(target)
C:\Users\navde\AppData\Lo

[['process', 'triacylglycerol biosynthesis'], ['CHEMICAL', 'TG(21:0/22:0/a-17:0'], ['CHEMICAL', 'triacylglycerol'], ['process', 'lipid metabolism'], ['role', 'energy storage']]
[['process', 'triacylglycerol biosynthesis'], ['CHEMICAL', 'TG(21:0/22:0/a-17:0'], ['CHEMICAL', 'triacylglycerol'], ['process', 'lipid metabolism'], ['role', 'energy storage']]
[['organoleptic_effect', 'Acid'], ['process', 'Cardiolipin Biosynthesis'], ['organoleptic_effect', 'Acid'], ['CHEMICAL', 'Cardiolipin'], ['CHEMICAL', 'Cardiolipin'], ['process', 'oxidative phosphorylation'], ['CHEMICAL', 'cardiolipin'], ['organoleptic_effect', 'acid'], ['organoleptic_effect', 'acid'], ['process', 'cardiolipin biosynthesis'], ['organoleptic_effect', 'acid'], ['CHEMICAL', 'cardiolipin'], ['process', 'cardiolipin biosynthesis'], ['organoleptic_effect', 'acid'], ['CHEMICAL', 'cardiolipin'], ['organoleptic_effect', 'acid'], ['organoleptic_effect', 'fatty'], ['organoleptic_effect', 'acid'], ['process', 'cardiolipin biosynthesis

...

c:\Users\navde\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\navde\AppData\Local\Temp\ipykernel_9356\3267912990.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output, target = torch.tensor(output), torch.tensor(target)
C:\Users\navde\AppData\Local\Temp\ipykernel_9356\3267912990.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output, target = torch.tensor(output), torch.tensor(target)
c:\Users\navde\AppData\Local\Progr

[['CHEMICAL', 'Cardiolipin'], ['process', 'Triacylglycerol Biosynthesis'], ['organoleptic_effect', 'acid'], ['role', 'essential'], ['process', 'lipid metabolism'], ['disposition', 'membrane'], ['CHEMICAL', 'cardiolipin'], ['process', 'triacylglycerol biosynthesis'], ['CHEMICAL', 'Cardiolipin'], ['role', 'essential'], ['disposition', 'inner mitochondrial <disposition>membrane'], ['CHEMICAL', 'triacylglycerols'], ['disposition', 'adipose tissue'], ['role', 'energy source'], ['CHEMICAL', 'cardiolipin'], ['process', 'triacylglycerol biosynthesis'], ['disposition', 'cell'], ['CHEMICAL', 'cardiolipin'], ['process', 'triacylglycerol biosynthesis'], ['CHEMICAL', 'LysoPA'], ['process', 'cardiolipin biosynthesis'], ['CHEMICAL', 'cardiolipin'], ['CHEMICAL', 'LysoPA'], ['process', 'triacylglycerol biosynthesis'], ['CHEMICAL', 'cardiolipin'], ['process', 'triacylglycerol biosynthesis'], ['CHEMICAL', 'LysoPA'], ['process', 'homeostasis'], ['process', 'lipid metabolism']]
[['CHEMICAL', 'LysoPA'], ['p

c:\Users\navde\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\navde\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\navde\AppData\Local\Temp\ipykernel_9356\3267912990.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output, target = torch.tensor(output), torch.tensor(target)
C:\Users\navde\AppData\Local\Temp\i

F1 Score:
  Average: 0.6997089248895645
  Minimum: 0.2142857164144516
  Maximum: 1.0
  Standard Deviation: 0.2974881625214022
Precision:
  Average: 0.699708921611045
  Minimum: 0.21428571428571427
  Maximum: 1.0
  Standard Deviation: 0.29748816562988983
Recall:
  Average: 0.699708921611045
  Minimum: 0.21428571428571427
  Maximum: 1.0
  Standard Deviation: 0.29748816562988983
Jaro-Winkler Score:
  Average: 0.883356303826572
  Minimum: 0.6854723506981075
  Maximum: 1.0
  Standard Deviation: 0.11537571556630502
Successful Outputs: 50
Failed Outputs: 0
Total Outputs: 50


FileNotFoundError: [Errno 2] No such file or directory: 'Tagger/results_of_tagger_bias_2.txt'